In [1]:
import time
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from meerkat import bme680, tools
from meerkat.base import TimePiece

In [3]:
from importlib import reload

In [4]:
def bp(x):
    return tools.bprint(x, n=8)

In [5]:
tp = TimePiece()

In [6]:
reload(bme680)
dev = bme680.BME680(bus_n=1)

In [7]:
dev.read_calibration()

1. Set humidty oversample
2. Set temperature oversample
3. Set pressure oversample

In [8]:
dev.set_oversampling(h=1, t=2, p=16)

4. Set gas wait time with gas_wait_0 to 0x59 = 100ms

In [9]:
wait = dev.calc_wait_time(t=25, x=4)
print(wait, hex(wait))

89 0x59


In [10]:
dev.set_gas_wait(n=0, value=wait)

5. Set heater set-point with res_heat_0, 150 C here

In [11]:
resistance = dev.calc_res_heat(target_temp=150)
resistance

151

In [12]:
dev.set_res_heat(n=0, value=resistance)

6. Set nb_conv to the 0 profile used in steps 4 and 5

In [13]:
dev.nb_conv = 0

7. Set run_gas to 1 to enable gas measurements

In [14]:
dev.gas_on()

8. Set mode to 0b01 to trigger a single forced mode measurement

In [15]:
dev.forced_mode()

In [16]:
%%time
dev.measure()
t = dev.temperature()
p = dev.pressure()
h = dev.humidity()
print("T: ", t)
print("P: ", p)
print("RH:", h)
#dev.setup_gas(t_ms=40, x=4, t_c=150)

T:  25.004609375
P:  999.3404630221833
RH: 31.543904546490346
CPU times: user 36.7 ms, sys: 2.37 ms, total: 39 ms
Wall time: 351 ms


In [17]:
%%time
t = dev.temperature()
p = dev.pressure()
h = dev.humidity()

CPU times: user 128 µs, sys: 18 µs, total: 146 µs
Wall time: 183 µs


In [18]:
print("T: ", t)
print("P: ", p)
print("RH:", h)

T:  25.004609375
P:  999.3404630221833
RH: 31.543904546490346


In [19]:
dev._range_switch_error

243

In [20]:
%%time
g = dev.gas()

CPU times: user 66 µs, sys: 9 µs, total: 75 µs
Wall time: 108 µs


In [21]:
g

(30669.599606688604, 563, 8, 83722239, 85393407)

In [ ]:
hrs = 6   # hours
delay = 2 # seconds
data = []
c = 0
while True:
    dev.forced_mode()
    dev.measure()
    t = dev.temperature()
    p = dev.pressure()
    h = dev.humidity()
    
    g = dev.gas()
    d = [tp.std_time(), t,p,h] + list(g) + [dev._gas_valid, dev._heat_stab]
    print(d)
    data.append(d)
    time.sleep(delay)
    if c > ((hrs * 60 * 60 )/ delay):
        break

['2020-03-25 00:44:46', 26.1286328125, 999.3698282410176, 31.605636265300255, 33996.107688876, 308, 8, 83722239, 77037567, 1, 3]
['2020-03-25 00:44:49', 25.288203125, 999.35573749439, 31.056647313375382, 36018.37910692307, 176, 8, 83722239, 72712191, 1, 3]
['2020-03-25 00:44:51', 25.2856640625, 999.3391740303093, 30.71946489598359, 37540.97617882963, 86, 8, 83722239, 69763071, 1, 3]
['2020-03-25 00:44:54', 25.3008984375, 999.3752728634992, 30.53633108042905, 54771.28071618692, 893, 7, 83052462, 95537070, 1, 3]
['2020-03-25 00:44:56', 25.3059765625, 999.3380850660121, 30.357964650690363, 56736.72073421223, 792, 7, 83052462, 92227502, 1, 3]
['2020-03-25 00:44:58', 25.3161328125, 999.3480873659438, 30.22352488928452, 58439.28906001736, 710, 7, 83052462, 89540526, 1, 3]
['2020-03-25 00:45:01', 25.331171875, 999.3627629785916, 30.13827669669474, 60133.76659460827, 633, 7, 83052462, 87017390, 1, 3]
['2020-03-25 00:45:03', 25.33625, 999.3501842561451, 30.09546187888622, 61571.27431849947, 571

In [ ]:
bp(0xF0//16)

In [ ]:
df = pd.DataFrame(data, columns=["dt", "t", "p", "h", 
                                 "gas_res"]) #, "adc_gas", "adc_gas2", "v1", "v2"])
df.dt = pd.to_datetime(df.dt)

In [ ]:
df[["dt", "gas_res"]].plot(x="dt");

In [ ]:
reload(bme680)
dev = bme680.BME680(bus_n=1)
dev.debug_read_registers()